# Database Lifecycle Report (BMC Discovery)

This notebook reproduces the DisMAL `db_lifecycle` report using the Discovery Data API.
It reads configuration from `../config.yaml`, executes the TWQL that DisMAL uses,
and writes a CSV to the same output structure as the CLI.

## Requirements

We use `requests` for HTTP, `pandas` for tabular data, and `PyYAML` to read configuration.
Uncomment the cell below to install them if needed.

In [ ]:
# %pip install -q requests pandas pyyaml

import pandas as pd
import requests
import yaml
from pathlib import Path
from urllib.parse import urljoin
import json, os

## Select Appliance (optional)

If your `config.yaml` defines multiple appliances under the `appliances:` list,
set `APPLIANCE_NAME` to one of their names or use the numeric index.
Defaults to the first appliance if neither is set.

In [ ]:
APPLIANCE_NAME = None   # e.g., 'prod' or 'dev'
APPLIANCE_INDEX = 0     # integer index if not using name selection

## Configuration (from config.yaml)

Reads settings from `../config.yaml` including target, token/token_file,
API version, and SSL verification preference.
Saves the CSV to `../output_<target>/db_lifecycle.csv`.

In [ ]:
# Robustly locate the project root (directory containing config.yaml)
def _find_repo_root(start: Path) -> Path:
    for p in [start] + list(start.parents):
        if (p / 'config.yaml').exists():
            return p
    return start.parent

repo_root = _find_repo_root(Path.cwd())
config_path = repo_root / 'config.yaml'
with open(config_path, 'r') as fh:
    cfg = yaml.safe_load(fh) or {}

apps = cfg.get('appliances') or []
selected = None
if isinstance(apps, list) and apps:
    if APPLIANCE_NAME:
        selected = next((a for a in apps if a.get('name') == APPLIANCE_NAME), None)
        if selected is None:
            raise ValueError(f"No appliance named '{APPLIANCE_NAME}' in config.yaml")
    else:
        try:
            selected = apps[int(APPLIANCE_INDEX)]
        except Exception:
            selected = apps[0]

target = ((selected or {}).get('target') or cfg.get('target') or '').strip()
if not target:
    raise ValueError('config.yaml missing "target"')
BASE_URL = target if ('://' in target) else f'https://{target}'

token = (((selected or {}).get('token') or cfg.get('token') or '').strip())
token_file = (selected or {}).get('token_file') or cfg.get('token_file') or cfg.get('f_token')
if not token and token_file:
    tf_path = Path(token_file)
    if not tf_path.is_absolute():
        tf_path = repo_root / tf_path
    with open(tf_path, 'r') as tf:
        token = tf.read().strip()
if not token:
    raise ValueError('API token not found in config.yaml (token or token_file)')

API_VERSION = str((selected or {}).get('api_version') or cfg.get('api_version') or 'v1.14')
VERIFY_SSL = bool((selected or {}).get('verify_ssl', cfg.get('verify_ssl', True)))

sanitized = target.replace('.', '_').replace(':', '_').replace('/', '_')
output_dir = repo_root / f'output_{sanitized}'
output_dir.mkdir(parents=True, exist_ok=True)

print('Appliance     :', (selected or {}).get('name', '(single)'))
print('Base URL      :', BASE_URL)
print('API Version   :', API_VERSION)
print('Verify SSL    :', VERIFY_SSL)
print('Output folder :', output_dir)

## Session and helpers

Create a session with Authorization header and helpers to call the Data API.
- `api_url(path)` builds endpoint URLs
- `normalize_results(raw)` flattens list/table responses
- `post_search(query, limit=0)` executes TWQL with pagination when needed

In [ ]:
session = requests.Session()
auth_value = token if token.lower().startswith('bearer ') else f'Bearer {token}'
session.headers.update({'Authorization': auth_value, 'Accept': 'application/json'})
session.verify = VERIFY_SSL

def api_url(path: str) -> str:
    base = BASE_URL.rstrip('/') + f'/api/{API_VERSION}/'
    return urljoin(base, path.lstrip('/'))

def normalize_results(raw):
    rows = []
    if isinstance(raw, dict):
        res = raw.get('results')
        if isinstance(res, dict):
            headers = res.get('headers') or res.get('columns')
            rws = res.get('rows') or res.get('data')
            if isinstance(headers, list) and isinstance(rws, list):
                return [dict(zip(headers, r)) for r in rws]
        if isinstance(res, list):
            rows = res
        elif isinstance(raw, list):
            rows = raw
        else:
            rows = []
    elif isinstance(raw, list):
        rows = raw
    else:
        rows = []
    if rows and isinstance(rows[0], list):
        headers = rows[0]
        return [dict(zip(headers, r)) for r in rows[1:]]
    return rows

def post_search(query: str, *, limit: int | None = None, page_size: int = 500):
    url = api_url('data/search')
    rows_all = []
    offset = 0
    fetch_all = (limit == 0)
    while True:
        payload = {'query': query, 'format': 'object'}
        if fetch_all:
            payload['limit'] = page_size
            if offset:
                payload['offset'] = offset
        elif limit is not None:
            payload['limit'] = limit
        r = session.post(url, json=payload)
        if r.status_code >= 400:
            print(f'Error {r.status_code} POST {url}: {r.text[:200]}')
            try:
                data = r.json()
            except Exception:
                data = []
            return normalize_results(data)
        try:
            data = r.json()
        except Exception:
            data = []
        rows = normalize_results(data)
        if not fetch_all:
            return rows
        rows_all.extend(rows)
        if not rows or len(rows) < page_size:
            break
        offset += page_size
    return rows_all

## TWQL for Database Lifecycle

This query mirrors DisMAL’s `queries.db_lifecycle`.

In [ ]:
qry_db_lifecycle = '''
search Pattern
where 'Relational Database Management Systems' in categories
traverse Pattern:Maintainer:Element:SoftwareInstance
    where #ElementWithDetail:SupportDetail:SoftwareDetail:SupportDetail.retirement_date
       or #ElementWithDetail:SupportDetail:SoftwareDetail:SupportDetail.end_support_date
       or #ElementWithDetail:SupportDetail:SoftwareDetail:SupportDetail.end_ext_support_date
show
    type as "SoftwareInstance.type",
    product_version as "SoftwareInstance.product_version",
    (#ElementWithDetail:SupportDetail:SoftwareDetail:SupportDetail.retirement_date
        and formatTime(#ElementWithDetail:SupportDetail:SoftwareDetail:SupportDetail.retirement_date, '%Y-%m-%d')) as 'End of Life',
    (#ElementWithDetail:SupportDetail:SoftwareDetail:SupportDetail.end_support_date
        and formatTime(#ElementWithDetail:SupportDetail:SoftwareDetail:SupportDetail.end_support_date, '%Y-%m-%d')) as 'End of Support',
    (#ElementWithDetail:SupportDetail:SoftwareDetail:SupportDetail.end_ext_support_date
        and formatTime(#ElementWithDetail:SupportDetail:SoftwareDetail:SupportDetail.end_ext_support_date, '%Y-%m-%d')) as 'End of Ext Support',
    (#ElementWithDetail:SupportDetail:SoftwareDetail:SupportDetail.end_ext_support_date
        and (#ElementWithDetail:SupportDetail:SoftwareDetail:SupportDetail.end_ext_support_date < currentTime() and 'EOES Exceeded')
        or #ElementWithDetail:SupportDetail:SoftwareDetail:SupportDetail.end_support_date
            and (#ElementWithDetail:SupportDetail:SoftwareDetail:SupportDetail.end_support_date < currentTime() and 'EOS Exceeded')
            or #ElementWithDetail:SupportDetail:SoftwareDetail:SupportDetail.retirement_date
                and (#ElementWithDetail:SupportDetail:SoftwareDetail:SupportDetail.retirement_date < currentTime() and 'EOL Exceeded')
                or (#ElementWithDetail:SupportDetail:SoftwareDetail:SupportDetail.retirement_date and (#ElementWithDetail:SupportDetail:SoftwareDetail:SupportDetail.retirement_date < currentTime() + 182 * 864000000000 and 'EOL less than 6 months away')
                    or #ElementWithDetail:SupportDetail:SoftwareDetail:SupportDetail.end_support_date and (#ElementWithDetail:SupportDetail:SoftwareDetail:SupportDetail.end_support_date < currentTime() + 182 * 864000000000 and 'EOS less than 6 months away')
                    or #ElementWithDetail:SupportDetail:SoftwareDetail:SupportDetail.end_ext_support_date and (#ElementWithDetail:SupportDetail:SoftwareDetail:SupportDetail.end_ext_support_date < currentTime() + 182 * 864000000000 and 'EOES less than 6 months away'))
                or (#ElementWithDetail:SupportDetail:SoftwareDetail:SupportDetail.retirement_date and 'EOL more than 6 months away'
                    or #ElementWithDetail:SupportDetail:SoftwareDetail:SupportDetail.end_support_date and 'EOS more than 6 months away'
                    or #ElementWithDetail:SupportDetail:SoftwareDetail:SupportDetail.end_ext_support_date and 'EOES more than 6 months away')) as 'Lifecycle Risk'
'''

## Execute query and load into pandas

We request all rows (limit=0) with pagination and build a DataFrame.
Columns are ordered to match DisMAL’s CSV header behavior (sorted keys)
with 'Discovery Instance' inserted as the first column.

In [ ]:
qry_results = post_search(qry_db_lifecycle, limit=0)
if not isinstance(qry_results, list):
    qry_results = []

#print(qry_results)

df = pd.DataFrame(qry_results[0]["results"], columns=qry_results[0]["headings"])
display(df.head(10))

df.insert(0, 'Discovery Instance', target)
df.head()

## Save to CSV

Writes the CSV to the standard CLI output folder.

In [ ]:
OUTPUT_CSV = str(output_dir / 'db_lifecycle.csv')
df.to_csv(OUTPUT_CSV, index=False)
print(f'Saved to {OUTPUT_CSV}')